In [3]:
pip install pygubu
pip install opencv-python 
pip install tk

SyntaxError: invalid syntax (3374645178.py, line 1)

In [9]:
pip install pygubu

Note: you may need to restart the kernel to use updated packages.


In [5]:
#import tkinter as tk
import tk
import ttk
#from tkinter import ttk
import SimpleITK as sitk
from PIL import Image, ImageTk
from pygubu import Builder
import cv2
import numpy as np

class SegmentationViewer:
    def __init__(self, images, segmentations, orientation):
        
        self.segmentations = segmentations
        
        self.ct = images['ct']
        self.pet = images['pet']
        
        self.current_segmentation = 0
        self.current_slice = 0
        
        self.orientation = orientation
        
        self.root = tk.Tk()
        self.root.title("Segmentation Viewer")
        
        self.image_label = tk.Label(self.root, width=600, height=600)
        self.image_label.pack()
        
        tk.Button(self.root, text="Previous", command=self.previous_segmentation).pack()
        tk.Button(self.root, text="Next", command=self.next_segmentation).pack()
        
        self.item_var = tk.StringVar(self.root)
        self.item_var.set(list(self.segmentations.keys())[0])
        self.item_dropdown = ttk.Combobox(self.root, textvariable=self.item_var, values=list(self.segmentations.keys()))
        self.item_dropdown.pack()
        self.item_dropdown.bind('<<ComboboxSelected>>', self.show_segmentation)
        
        self.slice_var = tk.IntVar(self.root)
        self.slice_var.set(0)
        self.slice_slider = tk.Scale(self.root, from_=0, to=143, orient='horizontal', variable=self.slice_var, command=self.show_slice)
        self.slice_slider.pack()
        
        self.show_segmentation()
        self.root.mainloop()
        
    def previous_segmentation(self):
        self.current_segmentation = (self.current_segmentation - 1) % len(self.segmentations[self.item_var.get()])
        self.show_segmentation()
        
    def next_segmentation(self):
        self.current_segmentation = (self.current_segmentation + 1) % len(self.segmentations[self.item_var.get()])
        self.show_segmentation()
        
    def show_slice(self, value):
        self.current_slice = int(value)
        self.show_segmentation()
    
    def show_segmentation(self, event=None):
        
        if self.orientation == 'x':
            ct_image = self.ct[self.current_slice,:,:]
            segmentation = self.segmentations[self.item_var.get()][self.current_slice,:,:]
        elif self.orientation == 'y':
            ct_image = self.ct[:,self.current_slice,:]
            segmentation = self.segmentations[self.item_var.get()][:,self.current_slice,:]
        elif self.orientation == 'z':
            ct_image = self.ct[:,:,self.current_slice]
            segmentation = self.segmentations[self.item_var.get()][:,:,self.current_slice]
        
        #ct_image = cv2.cvtColor(ct_image, cv2.COLOR_BGR2GRAY)
        
        overlayed_image = cv2.addWeighted(ct_image, 0.5, segmentation, 0.5, 0)
        overlayed_image = Image.fromarray(overlayed_image)
        overlayed_image = overlayed_image.resize((500,500), Image.ANTIALIAS)
        overlayed_image = ImageTk.PhotoImage(overlayed_image)
        self.image_label.config(image=overlayed_image)
        self.image_label.image = overlayed_image
        
        #image = Image.fromarray(segmentation)
        #image = ImageTk.PhotoImage(image)
        #self.image_label.config(image=image)
        #self.image_label.image = image


ModuleNotFoundError: No module named 'ttk'

In [ ]:
path_output = r'\\zkh\appdata\RTDicom\HANARTHproject\Baoqiang\UMCG_outpredL_resampled'

patient_ID = 'UMCG-1117043'

CT_image = sitk.ReadImage(path_output+'/'+patient_ID+"_ct.nii.gz")
vol_ct = sitk.GetArrayFromImage(CT_image)
vol_ct = vol_ct.clip(-200,200)

PET_image = sitk.ReadImage(path_output+'/'+patient_ID+"_pt.nii.gz")
vol_pet = sitk.GetArrayFromImage(PET_image)

PT_image = sitk.ReadImage(path_output+'/'+patient_ID+"_ct_GTVpt.nii.gz")
vol_pt = sitk.GetArrayFromImage(PT_image)
vol_pt = np.ma.masked_where(vol_pt==False,vol_pt)

LN_image = sitk.ReadImage(path_output+'/'+patient_ID+"_ct_GTVln.nii.gz")
vol_ln = sitk.GetArrayFromImage(LN_image)
vol_ln = np.ma.masked_where(vol_ln==False,vol_ln)

images = {'ct': vol_ct, 'pet': vol_pet}
segmentations = {'primary tumor': vol_pt, 'lymph node': vol_ln}

In [ ]:
viewer = SegmentationViewer(images, segmentations,'x')